In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import CosineRecommender, bm25_weight
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix

In [2]:
data_dir = "/Users/ur001/Documents/Datasets/lastfm-dataset-360K"
dest_data_dir = "/Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest"

In [3]:
def load_data(path): 
    return pd.read_table(
        os.path.join(path, "usersha1-artmbid-artname-plays.tsv"),
        usecols=[0, 1, 2, 3],
        names=['user', 'artist', 'artist_name', 'plays'],
    )

def prepare_data(data):
    """Возвращает подготовленные данные и словарь артистов"""
    # Удаляем записи пользователей встречающихся только 1 раз
    print("Removing user with only one record...")
    data = data.loc[data.duplicated(subset=['user'], keep=False), :]
    
    # Заменяем отсутствующие значения идентификаторов исполнителя на его имя
    print("Filling na artists...")
    data = data.dropna(subset=['artist_name'])
    empty_artist = data.artist.isnull()
    data.loc[empty_artist, 'artist'] = data.loc[empty_artist, 'artist_name']
    
    # Преобразуем пользователей и исполнителей в числа
    print("Encoding artits and users...")
    data['artist_id'] = data.artist.astype("category").cat.codes.copy() + 1
    data['user_id'] = data.user.astype("category").cat.codes.copy() + 1
    
    # Составляем словарь исполнителей
    print("Saving atists dict...")
    artists = dict(data.groupby('artist_id').artist_name.first())
    return data.loc[:, ['user_id', 'artist_id', 'plays']], artists

In [44]:
def split_data(data, size=0.2):
    data_train, data_test = train_test_split(data, test_size=size, stratify=data.user_id, random_state=0)
    test_user_set = set(data_test.user_id.unique())
    train_user_set = set(data_train.user_id.unique())
    test_artist_set = set(data_test.artist_id.unique())
    train_artist_set = set(data_train.artist_id.unique())    
    
    # Оставляем только пользователей которые есть одновременно в тестовой и обучающей выборке
    # Так же удаляем из тестовой выборки записи о неизвестных в тренировочной выборке исполнителях
    user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
    artist_ids_to_exclude = (test_artist_set - train_artist_set)
    return (
        data_train[~data_train.user_id.isin(user_ids_to_exclude)], 
        data_test[~data_test.user_id.isin(user_ids_to_exclude) & ~data_test.artist_id.isin(artist_ids_to_exclude)]
    )

In [45]:
def save_data(path, name, data, columns=["user_id", "artist_id", "plays"]):
    file_name = os.path.join(path, name + ".tsv")
    data[columns].to_csv(file_name, sep="\t", header=False, index=False)

In [46]:
def make_coo_matrix(data):
    """Создаёт разреженную матрицу item*user"""
    return coo_matrix((
        data.plays.astype(np.double),
        (data.artist_id, data.user_id)
    ))

def sparse_info(sparse_matrix):
    """функция, которая красиво печатает информацию о разреженных матрицах"""
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}".format(
        sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1]
    ))
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [47]:
def make_user_recommendations(model, data_train_coo, data_test, path, model_name):
    """Генерирует рекомендации для каждого пользователя и сохраняет их в файл для проверки mrec-ом"""
    user_plays = data_train_coo.T.tocsr()
    file_name = "test.{}.tsv.recs.tsv".format(model_name)
    # Получаем столько рекомендаций, сколько записей у пользователя в тестовой выборке, но не меньше 3
    user_rec_counts = dict(data_test.user_id.value_counts())
    user_rec_counts = {user_id: max(10, int(N)) for user_id, N in user_rec_counts.items()}
    with open(os.path.join(path, file_name), "w") as output_file:
        for user_id in tqdm_notebook(data_test.user_id.unique()):
            for artist_id, score in model.recommend(user_id, user_plays, N=user_rec_counts[user_id]):
                output_file.write("{}\t{}\t{}\n".format(user_id, artist_id, score))    

In [48]:
def search_artist(artists, name_prefix):
    """Поиск по базе исполнителей: выводит список всех начинающихся с переданного префикса"""
    for artist_id, artist_name in artists.items():
        if artist_name.lower().startswith(name_prefix):
            print("[{}]\t{}".format(artist_id, artist_name))
            
def find_similar_artists(model, artist_id, artists, count=10):
    """Выводит список похожих исполнителей"""
    print("Similar to {}:".format(artists[artist_id]))
    print("-------------------------------")
    for similar_artist_id, score in model.similar_items(artist_id, count):
        print("{:.3f}\t{}".format(score, artists[similar_artist_id]))

In [9]:
data_src = load_data(data_dir)

In [140]:
data, artists = prepare_data(data_src)

Removing user with only one record...
Filling na artists...
Encoding artits and users...
Saving atists dict...


In [141]:
data.head()

,user_id,artist_id,plays
0,1,39710,2137
1,1,211103,1099
2,1,135573,897
3,1,40727,717
4,1,143197,706


In [142]:
data_train, data_test = split_data(data, 0.2)
data_train.shape, data_test.shape

((14028062, 3), (3480715, 3))

In [143]:
data_train.head()

,user_id,artist_id,plays
12187595,249380,100541,12
13826992,282926,28478,88
14533898,297377,39001,111
4198859,85897,215673,202
14778877,302392,57928,128


In [144]:
save_data(dest_data_dir, 'train', data_train)

In [145]:
data_train_coo = make_coo_matrix(data_train)

In [146]:
sparse_info(data_train_coo)

Размерности матрицы: (268584, 358834)
Ненулевых элементов в матрице: 14028062
Доля ненулевых элементов: 0.00014555393566431353
Среднее значение ненулевых элементов: 215.06875874942668
Максимальное значение ненулевых элементов: 419157.0
Минимальное значение ненулевых элементов: 0.0


In [147]:
data_train_csr = data_train_coo.tocsr()
data_train_bm25 = bm25_weight(data_train_coo, K1=100, B=0.8).tocsr()

In [148]:
# строим матрицу схожести по косинусной мере
models = {}
models['knn_cosine'] = CosineRecommender(K=500)
models['knn_cosine'].fit(data_train_bm25)

In [149]:
save_data(dest_data_dir, 'test.knn_cosine', data_test)

In [150]:
# Попробуем найти похожих исполнителей на Aphex Twin, чтобы убедиться что рекомендации вообще работают
search_artist(artists, 'aphex twin')

[128601]	aphex twin & céline eia
[128602]	aphex twin & m-ziq (aka rich & mike)
[128603]	aphex twin & squarepusher
[128604]	aphex twin & µ-ziq (mike & rich)
[128605]	aphex twin & µ-ziq - mike and rich
[128606]	aphex twin (aka afx)
[128607]	aphex twin (aka caustic window)
[128608]	aphex twin (aka polygon window)
[128609]	aphex twin (aka universal indicator)
[128610]	aphex twin - power-pill
[128611]	aphex twin ft. photek & autechre
[128612]	aphex twin vs luke vibert
[128613]	aphex twin vs run jeremy
[128614]	aphex twin/afx
[128615]	aphex twin/luke vibert
[128616]	aphex twin/richard james
[210590]	aphex twin


In [151]:
aphex_twin_id = 210590
find_similar_artists(models['knn_cosine'], aphex_twin_id, artists)

Similar to aphex twin:
-------------------------------
1.000	aphex twin
0.373	afx
0.309	squarepusher
0.264	autechre
0.245	boards of canada
0.209	polygon window
0.202	venetian snares
0.193	the tuss
0.184	plaid
0.156	bogdan raczynski


In [152]:
# Создаём рекомендации для всех тестовых пользователей
make_user_recommendations(models['knn_cosine'], data_train_csr, data_test, dest_data_dir, 'knn_cosine')

HBox(children=(IntProgress(value=0, max=358704), HTML(value='')))

In [153]:
!/Users/ur001/.pyenv/versions/netology1/bin/mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest/test.knn_cosine.tsv \
    --recsdir /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest

[2017-12-05 17:32:35,593] INFO: processing /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest/test.knn_cosine.tsv...
None
mrr            0.4197 +/- 0.0000
prec@5         0.1880 +/- 0.0000
prec@10        0.1433 +/- 0.0000
prec@15        0.0979 +/- 0.0000
prec@20        0.0735 +/- 0.0000


In [154]:
# Тренируем ALS-модель
models['als'] = AlternatingLeastSquares(factors=256, iterations=15, regularization=0.001)
models['als'].fit(data_train_bm25)

In [155]:
find_similar_artists(models['als'], aphex_twin_id, artists)

Similar to aphex twin:
-------------------------------
1.000	aphex twin
0.968	boards of canada
0.935	squarepusher
0.933	autechre
0.899	amon tobin
0.889	afx
0.885	venetian snares
0.875	massive attack
0.873	burial
0.873	m83


In [156]:
save_data(dest_data_dir, 'test.als', data_test)

In [157]:
make_user_recommendations(models['als'], data_train_bm25, data_test, dest_data_dir, 'als')

HBox(children=(IntProgress(value=0, max=358704), HTML(value='')))

In [158]:
!/Users/ur001/.pyenv/versions/netology1/bin/mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest/test.als.tsv \
    --recsdir /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest

# РЕЗУЛЬТАТЫ ЭКСПЕРИМЕНТОВ НА УРЕЗАННОМ ДАТАСЕТЕ 300000 записей

# factors=128, iterations=100, разная регуляризация
# models['als'] = AlternatingLeastSquares(factors=128, iterations=100, regularization=0.1)
# mrr            0.3464 +/- 0.0000
# prec@5         0.1503 +/- 0.0000
# prec@10        0.1198 +/- 0.0000
# prec@15        0.0813 +/- 0.0000
# prec@20        0.0610 +/- 0.0000


# models['als'] = AlternatingLeastSquares(factors=128, iterations=100, regularization=0.01)
# mrr            0.3463 +/- 0.0000
# prec@5         0.1510 +/- 0.0000
# prec@10        0.1209 +/- 0.0000
# prec@15        0.0820 +/- 0.0000
# prec@20        0.0615 +/- 0.0000

# models['als'] = AlternatingLeastSquares(factors=128, iterations=100, regularization=0.001)
# mrr            0.3452 +/- 0.0000
# prec@5         0.1511 +/- 0.0000
# prec@10        0.1200 +/- 0.0000
# prec@15        0.0817 +/- 0.0000
# prec@20        0.0613 +/- 0.0000

# models['als'] = AlternatingLeastSquares(factors=128, iterations=100, regularization=0.0001)
# mrr            0.3475 +/- 0.0000
# prec@5         0.1524 +/- 0.0000
# prec@10        0.1198 +/- 0.0000
# prec@15        0.0815 +/- 0.0000
# prec@20        0.0611 +/- 0.0000

# factors=128, iterations=2, разная регуляризация
# models['als'] = AlternatingLeastSquares(factors=128, iterations=2, regularization=0.0001)
# mrr            0.1342 +/- 0.0000
# prec@5         0.0528 +/- 0.0000
# prec@10        0.0450 +/- 0.0000
# prec@15        0.0306 +/- 0.0000
# prec@20        0.0229 +/- 0.0000

# models['als'] = AlternatingLeastSquares(factors=128, iterations=2, regularization=0.1)
# mrr            0.1456 +/- 0.0000
# prec@5         0.0568 +/- 0.0000
# prec@10        0.0485 +/- 0.0000
# prec@15        0.0330 +/- 0.0000
# prec@20        0.0248 +/- 0.0000

# factors=64, iterations=100, разная регуляризация
# models['als'] = AlternatingLeastSquares(factors=64, iterations=100, regularization=0.1)
# mrr            0.2997 +/- 0.0000
# prec@5         0.1310 +/- 0.0000
# prec@10        0.1079 +/- 0.0000
# prec@15        0.0733 +/- 0.0000
# prec@20        0.0550 +/- 0.0000

# models['als'] = AlternatingLeastSquares(factors=64, iterations=100, regularization=0.0001)
# mrr            0.3049 +/- 0.0000
# prec@5         0.1315 +/- 0.0000
# prec@10        0.1080 +/- 0.0000
# prec@15        0.0734 +/- 0.0000
# prec@20        0.0551 +/- 0.0000

# factors=64, iterations=500, разная регуляризация
# models['als'] = AlternatingLeastSquares(factors=64, iterations=500, regularization=0.0001)
# mrr            0.3060 +/- 0.0000
# prec@5         0.1322 +/- 0.0000
# prec@10        0.1079 +/- 0.0000
# prec@15        0.0734 +/- 0.0000
# prec@20        0.0551 +/- 0.0000

# models['als'] = AlternatingLeastSquares(factors=64, iterations=500, regularization=0.01)
# mrr            0.3065 +/- 0.0000
# prec@5         0.1327 +/- 0.0000
# prec@10        0.1079 +/- 0.0000
# prec@15        0.0734 +/- 0.0000
# prec@20        0.0551 +/- 0.0000

# factors=256, разное число итераций
# models['als'] = AlternatingLeastSquares(factors=256, iterations=2, regularization=0.001)
# mrr            0.1622 +/- 0.0000
# prec@5         0.0642 +/- 0.0000
# prec@10        0.0533 +/- 0.0000
# prec@15        0.0363 +/- 0.0000
# prec@20        0.0272 +/- 0.0000

# models['als'] = AlternatingLeastSquares(factors=256, iterations=15, regularization=0.001)
# mrr            0.3532 +/- 0.0000
# prec@5         0.1539 +/- 0.0000
# prec@10        0.1196 +/- 0.0000
# prec@15        0.0812 +/- 0.0000
# prec@20        0.0609 +/- 0.0000

# models['als'] = AlternatingLeastSquares(factors=256, iterations=100, regularization=0.001)
# mrr            0.3560 +/- 0.0000
# prec@5         0.1564 +/- 0.0000
# prec@10        0.1218 +/- 0.0000
# prec@15        0.0825 +/- 0.0000
# prec@20        0.0619 +/- 0.0000

[2017-12-06 00:42:44,073] INFO: processing /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest/test.als.tsv...
None
mrr            0.3607 +/- 0.0000
prec@5         0.1612 +/- 0.0000
prec@10        0.1268 +/- 0.0000
prec@15        0.0868 +/- 0.0000
prec@20        0.0651 +/- 0.0000


In [43]:
from implicit.approximate_als import NMSLibAlternatingLeastSquares

In [45]:
models['als_nmslib'] = NMSLibAlternatingLeastSquares(factors=20, iterations=50)
models['als_nmslib'].fit(data_train_bm25)

In [ ]:
find_similar_artists(models['als_nmslib'], aphex_twin_id, artists)

In [ ]:
make_user_recommendations(models['als_nmslib'], data_train_bm25, data_test, dest_data_dir, 'als_nmslib')

In [ ]:
save_data(dest_data_dir, 'test.als_nmslib', data_test)

In [ ]:
!/Users/ur001/.pyenv/versions/netology1/bin/mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest/test.als_nmslib.tsv \
    --recsdir /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest